In [39]:

from typing import List

import eli5
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier




In [6]:
# plt.rcParams['figure.facecolor'] = 'white'

In [7]:
# %load_ext autoreload
# %autoreload 2

#### Custom Transformers

In [8]:
# Define custom transformer to compute ratio of two columns
class FraudFeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            handle_unknown: str = "ignore",
            unknown_value: float = 0.0,
            ratio_upper_threshold: float = 100,
            ratio_lower_threshold: float = 0,
            algorithm: str = "linear",
    ):
        self.handle_unknown = handle_unknown
        self.unknown_value = unknown_value
        self.ratio_upper_threshold = ratio_upper_threshold
        self.ratio_lower_threshold = ratio_lower_threshold
        self.algorithm = algorithm

        self.epsilon = 1e-4
        self.numeric_features = [
            'amount',
            'oldbalanceOrg',
            'newbalanceOrig',
            'oldbalanceDest',
            'newbalanceDest',
        ]

        self.medians = None
        self.means = None
        self.feature_names_out = None

    def _compute_ratio_(self, X: pd.DataFrame, x_col: str, y_col: str) -> pd.Series:
        ratios = X[x_col] / X[y_col]

        # return pd.Series(np.where(ratios < self.ratio_threshold, self.ratio_threshold, ratios))
        return ratios.apply(lambda x: np.where(x > self.ratio_upper_threshold, self.ratio_upper_threshold, x)).apply(
            lambda x: np.where(x < self.ratio_lower_threshold, self.ratio_lower_threshold, x))

    def fit(self, X, y=None) -> None:
        self.means = {feat: X[feat].mean() for feat in self.numeric_features}
        self.medians = {feat: X[feat].median() for feat in self.numeric_features}

        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:

        new_df = pd.DataFrame()

        for feat in self.numeric_features:
            if feat != 'amount':
                new_df[f'{feat}_amount_ratio'] = self._compute_ratio_(X, feat, 'amount')

            if self.algorithm == 'linear':
                new_df[f'{feat}_is_zero'] = (X[feat] > self.epsilon).astype(int)
                new_df[f'{feat}_is_bigger_mean'] = (X[feat] > self.means[feat]).astype(int)
                new_df[f'{feat}_is_bigger_median'] = (X[feat] > self.medians[feat]).astype(int)

        new_df['dest_is_bigger_org'] = (X['oldbalanceDest'] > X['oldbalanceOrg']).astype(int)

        self.feature_names_out = new_df.columns.tolist()

        return new_df

    def get_feature_names_out(self, input_features=None):
        return self.feature_names_out

In [9]:
class LogTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, handle_inf="median", fill_value=0):
        self.handle_inf = handle_inf
        self.fill_value = fill_value
        self.feature_names_out = None

    def fit(self, X, y=None):

        if self.handle_inf == "median":
            logged_X = np.log(X)
            self.fill_value = np.nanmedian(logged_X, axis=0)
        elif self.handle_inf == "mean":
            logged_X = np.log(X)
            self.fill_value = np.nanmean(logged_X, axis=0)
        elif self.handle_inf == "const":
            pass

        self.feature_names_out = [f"{feat}_log" for feat in X.columns]

        return self

    def transform(self, X):
        logged_X = np.log(X)

        return np.where(np.isinf(logged_X), np.expand_dims(self.fill_value, axis=0), logged_X)

    def get_feature_names_out(self, input_features=None):
        if input_features is not None:
            return input_features

        return self.feature_names_out


In [10]:
class TargetEncoderFixed(TargetEncoder):
    def get_feature_names_out(self, *arg, **kargs) -> List[str]:
        return self.feature_names_out_

#### Loading data

In [11]:
fraud = pd.read_csv(r'onlinefraud\onlinefraud.csv', nrows=500000)  #nrows=100000

In [12]:
d = fraud.head(1000)

In [13]:
data = fraud

#### Features

In [14]:
features_to_exclude = [
    'isFlaggedFraud',
    'nameOrig',
    'nameDest',
]

In [15]:
features_to_exclude

['isFlaggedFraud', 'nameOrig', 'nameDest']

In [16]:
target_features = ['isFraud', 'isFlaggedFraud']

target = "isFraud"

In [17]:
features = [col for col in data.columns if col not in target_features + features_to_exclude]

In [18]:
data.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [19]:
numeric_features = [
    "amount",
    "oldbalanceOrg",
    "newbalanceOrig",
    "oldbalanceDest",
    "newbalanceDest",
]

In [20]:
nominal_features = [
    "type",
]

In [21]:
data[features].head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0


In [22]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


#### Pipelines

In [23]:
linear_generator = ColumnTransformer(
    transformers=[
        ('ffg', FraudFeatureGenerator(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown="ignore"), nominal_features),
    ],
    remainder='drop'
)

linear_preprocessor = ColumnTransformer(
    transformers=[
        ('log', LogTransformer(handle_inf='const', fill_value=0), numeric_features),
        ('scl', StandardScaler(), numeric_features),
    ],
    remainder='drop'
)

liner_data_pipeline = ColumnTransformer(
    transformers=[
        ('generator', linear_generator, features),
        ('preprocessor', linear_preprocessor, features),
    ],
    remainder='drop'
)

# Define the pipeline
pipeline = Pipeline([
    ('general_proc', liner_data_pipeline),
    ('regressor', LogisticRegression())
])

In [24]:
tree_generator = ColumnTransformer(
    transformers=[
        ('ffg', FraudFeatureGenerator(algorithm='tree'), numeric_features),
        ('cat', TargetEncoderFixed(handle_unknown="median"), nominal_features),
    ],
    remainder='drop'
)

tree_preprocessor = ColumnTransformer(
    transformers=[
        ('imputer', SimpleImputer(fill_value=-1), numeric_features),
    ],
    remainder='drop'
)

tree_data_pipeline = ColumnTransformer(
    transformers=[
        ('generator', tree_generator, features),
        ('preprocessor', tree_preprocessor, features),
    ],
    remainder='drop'
)

# Define the pipeline
pipeline = Pipeline([
    ('general_proc', tree_data_pipeline),
    ('regressor', RandomForestClassifier())
])


##### test transformers

In [25]:
tree_generator.fit_transform(d, d['isFraud'])

array([[17.2908765 , 16.2908765 ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [11.39796597, 10.39796597,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.0403924 ],
       ...,
       [ 5.45425101,  4.45425101,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 7.7631187 ,  6.7631187 ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.23593371,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [26]:
tree_generator.get_feature_names_out()

array(['ffg__oldbalanceOrg_amount_ratio',
       'ffg__newbalanceOrig_amount_ratio',
       'ffg__oldbalanceDest_amount_ratio',
       'ffg__newbalanceDest_amount_ratio', 'ffg__dest_is_bigger_org',
       'cat__type'], dtype=object)

In [23]:
# generator.fit_transform(d)

In [24]:
# generator.get_feature_names_out()

In [25]:
# preprocessor.fit_transform(d)

In [26]:
pipeline.fit(d, d['isFraud'])

Pipeline(steps=[('general_proc',
                 ColumnTransformer(transformers=[('generator',
                                                  ColumnTransformer(transformers=[('ffg',
                                                                                   FraudFeatureGenerator(algorithm='tree'),
                                                                                   ['amount',
                                                                                    'oldbalanceOrg',
                                                                                    'newbalanceOrig',
                                                                                    'oldbalanceDest',
                                                                                    'newbalanceDest']),
                                                                                  ('cat',
                                                                                   TargetEncoderFixed(handle_unknown='median'),
                                                                                   ['type'])]),
                                                  ['step', 'type', 'amount',
                                                   'oldbalanceOrg',
                                                   'newbalanceOrig',
                                                   'oldbalanceDest',
                                                   'newbalanceDest']),
                                                 ('preprocessor',
                                                  ColumnTransformer(transformers=[('imputer',
                                                                                   SimpleImputer(fill_value=-1),
                                                                                   ['amount',
                                                                                    'oldbalanceOrg',
                                                                                    'newbalanceOrig',
                                                                                    'oldbalanceDest',
                                                                                    'newbalanceDest'])]),
                                                  ['step', 'type', 'amount',
                                                   'oldbalanceOrg',
                                                   'newbalanceOrig',
                                                   'oldbalanceDest',
                                                   'newbalanceDest'])])),
                ('regressor', RandomForestClassifier())])

In [27]:
eli5.explain_weights_df(pipeline[-1], feature_names=pipeline[:-1].get_feature_names_out())

,feature,weight,std
0,generator__ffg__oldbalanceOrg_amount_ratio,0.311519,0.311484
1,preprocessor__imputer__amount,0.162009,0.169020
2,preprocessor__imputer__oldbalanceDest,0.123631,0.182555
3,generator__ffg__oldbalanceDest_amount_ratio,0.116195,0.185613
4,preprocessor__imputer__oldbalanceOrg,0.079014,0.133722
5,generator__cat__type,0.064649,0.108893
6,generator__ffg__newbalanceDest_amount_ratio,0.045492,0.097958
7,preprocessor__imputer__newbalanceDest,0.044861,0.084029
8,generator__ffg__newbalanceOrig_amount_ratio,0.021930,0.096987
9,generator__ffg__dest_is_bigger_org,0.021797,0.064910


#### Models

In [28]:
dummy_mf = make_pipeline(
    DummyClassifier(strategy='most_frequent'),
)
dummy_strf = make_pipeline(
    DummyClassifier(strategy='stratified'),
)

In [29]:

lr = Pipeline([
    ('general_proc', liner_data_pipeline),
    ('regressor', LogisticRegression())
])

svm = Pipeline([
    ('general_proc', liner_data_pipeline),
    ('regressor', SVC())
])

mlp = Pipeline([
    ('general_proc', liner_data_pipeline),
    ('regressor', MLPClassifier())
])

In [30]:


ctb = Pipeline([
    ('general_proc', tree_data_pipeline),
    ('regressor', CatBoostClassifier())
])
xgb = Pipeline([
    ('general_proc', tree_data_pipeline),
    ('regressor', XGBClassifier())
])
lgbm = Pipeline([
    ('general_proc', tree_data_pipeline),
    ('regressor', LGBMClassifier())
])
rfc = Pipeline([
    ('general_proc', tree_data_pipeline),
    ('regressor', RandomForestClassifier())
])

In [31]:
pipelines = [
    # ("Dummy_mf", dummy_mf),
    # ("Dummy_strf", dummy_strf),
    ("LinReg", lr),
    # ("XGB", xgb),
    ("LGBM", lgbm),
    # ("RFC", rfc),
    # ("CatBoost", ctb),
]

#### Data split

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, stratify=data[target])

In [33]:
X_train.shape, X_test.shape

((400000, 7), (100000, 7))

In [34]:
y_train.value_counts()

0    399814
1       186
Name: isFraud, dtype: int64

In [35]:
y_test.value_counts()

0    99953
1       47
Name: isFraud, dtype: int64

In [36]:
dummy_mf.fit(X_train, y_train)
dummy_strf.fit(X_train, y_train)

Pipeline(steps=[('dummyclassifier', DummyClassifier(strategy='stratified'))])

In [42]:
model = mlp

In [ ]:
model.fit(X_train, y_train)

In [40]:
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))

print("Confusion matrix:")
print(cm)



C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\pandas\core\internals\blocks.py:352: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     99953
           1       0.92      0.51      0.66        47

    accuracy                           1.00    100000
   macro avg       0.96      0.76      0.83    100000
weighted avg       1.00      1.00      1.00    100000

Confusion matrix:
[[99951     2]
 [   23    24]]


In [41]:
y_pred = model.predict(X_train)
cm = confusion_matrix(y_train, y_pred)

print(classification_report(y_train, y_pred))

print("Confusion matrix:")
print(cm)

C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\pandas\core\internals\blocks.py:352: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    399814
           1       0.93      0.43      0.59       186

    accuracy                           1.00    400000
   macro avg       0.96      0.72      0.79    400000
weighted avg       1.00      1.00      1.00    400000

Confusion matrix:
[[399808      6]
 [   106     80]]


In [55]:
weights = eli5.explain_weights_df(model[-1], feature_names=model[:-1].get_feature_names_out())

In [56]:
weights

#### Advanced (don't run)

In [57]:
from notebooks.Dzim.data_mining.validation.training import estimate_multiple_models
from notebooks.Dzim.data_mining.validation.training import estimate_model

#### training
model_names, model_pipelines = zip(*pipelines)
metrics, weights = estimate_multiple_models(
    model_pipelines,
    lambda pipeline, _: estimate_model(
        pipeline,
        (X_train, X_test, y_train, y_test),
    ),
    model_names
)

C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\pandas\core\internals\blocks.py:352: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\pandas\core\internals\blocks.py:352: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\pandas\core\internals\blocks.py:352: RuntimeW

In [58]:
metrics

set       train                                                    test  \
metric f1_score cohen_kappa_score precision_score recall_score f1_score   
model                                                                     
LinReg    0.520             0.520           0.971        0.355    0.554   
LGBM      0.003             0.002           0.002        0.005    0.000   

set                                                    
metric cohen_kappa_score precision_score recall_score  
model                                                  
LinReg             0.554             1.0        0.383  
LGBM              -0.001             0.0        0.000

In [59]:
print(weights)

                                                        weight
model  feature                                                
LinReg generator__ffg__oldbalanceOrg_amount_ratio        5.341
       generator__cat__type_TRANSFER                     3.749
       generator__cat__type_CASH_OUT                     2.980
       generator__ffg__oldbalanceOrg_is_bigger_mean      2.849
       preprocessor__scl__oldbalanceOrg                  2.029
       preprocessor__scl__oldbalanceDest                 0.543
       generator__ffg__amount_is_bigger_mean             0.528
       preprocessor__scl__amount                         0.276
       generator__ffg__oldbalanceDest_is_bigger_median   0.267
       preprocessor__log__oldbalanceOrg                  0.151
       generator__ffg__amount_is_bigger_median           0.143
       generator__ffg__newbalanceDest_is_zero            0.114
       generator__ffg__newbalanceDest_amount_ratio       0.025
       generator__ffg__newbalanceDest_is_bigger_mean   

In [ ]:
for name, _ in pipelines:
    try:
        print("#" * 80)
        print(name)
        print(weights.loc[name])
    except KeyError:
        print('Not Supported')